In [20]:
import pandas as pd
import numpy as np

df = pd.read_csv('第二问编码后的数据.csv',engine='python')

# 删除已处理变量

df = df.drop(df[["carid"]], axis=1)
withdraw = df['withdrawDate']
df = df.drop(labels=['withdrawDate'], axis=1)
df.insert(39, 'withdraw', withdraw)

### 3.划分训练集和测试集

In [23]:
#划分训练集和测试集
from sklearn.model_selection import train_test_split

lst = [i for i in range(0,39)]

X=df.iloc[:,lst]
y=df.loc[:,"withdraw"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

print("X_train.shape:",X_train.shape)
print("X_test.shape:",X_test.shape)
print("y_train.shape:",y_train.shape)
print("y_test.shape:",y_test.shape)

X_train.shape: (7000, 39)
X_test.shape: (3000, 39)
y_train.shape: (7000,)
y_test.shape: (3000,)


### 5.归一化处理

In [24]:
# 数值型变量归一化处理

num_columns = ['pushPrice','serial','model','mileage','transferCount','seatings',
               'displacement','newprice','price','feature12-length','feature12-width',
              'feature12-height','feature12-volume','tradeTime-registerDate',
               'tradeTime-licenseDate','tradeTime-modelyear','tradeTime-feature13',
              'pull_pushTime','update_num','update_T_el','delta_price']
X_train_num = X_train.loc[:,num_columns]
X_test_num = X_test.loc[:,num_columns]

X_train_num.columns = num_columns

from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler().fit(X_train_num)
X_train_num = min_max_scaler.transform(X_train_num)
X_test_num = min_max_scaler.transform(X_test_num)

X_train.drop(labels=num_columns,axis=1,inplace=True)
X_test.drop(labels=num_columns,axis=1,inplace=True)

X_train_num = pd.DataFrame(X_train_num,index=X_train.index)
X_train = pd.concat([X_train,X_train_num],axis=1)

X_test_num = pd.DataFrame(X_test_num,index=X_test.index)
X_test = pd.concat([X_test,X_test_num],axis=1)

### 模型评价

In [30]:
def scores(y_pred, y_test):
    y_pred = list(y_pred)
    y_test = list(y_test)
    
    TP, TN, FP, FN = 0, 0, 0, 0
    for i in range(len(y_pred)):
        if y_pred[i] == 1 and y_test[i]==1:
            TP += 1
        elif y_pred[i] == 1 and y_test[i]==0:
            FP += 1
        elif y_pred[i] == 0 and y_test[i]==0:
            TN += 1
        else:
            FN += 1
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f1 = 2*precision*recall/(recall+precision)
    return precision, recall, f1

### 逻辑回归

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

def logistic_regression(X_train, X_test):
    logit = LogisticRegression(class_weight='balanced')
    logit.fit(X_train, y_train)

    y_pred = logit.predict(X_test)
    coef = logit.coef_
    intercept = logit.intercept_
    
    return y_pred, coef, intercept

y_pred1, coef, intercept = logistic_regression(X_train, X_test)

precision1, recall1, f11 = scores(y_pred1, y_test)

print("逻辑回归：")
print(precision1)
print(recall1)
print(f11)

### 随机森林

In [34]:
import pydotplus
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn import metrics
from openpyxl import load_workbook
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import RandomForestClassifier

def random_forest_classifier(X_train, X_test):
    forest = RandomForestClassifier(criterion='entropy', class_weight='balanced',n_estimators=100)
    forest.fit(X_train, y_train)

    y_pred = forest.predict(X_test)
    
    estimator = forest.estimators_
    
    return y_pred, estimator, forest

y_pred2, estimator, forest = random_forest_classifier(X_train, X_test)

precision2, recall2, f12 = scores(y_pred2, y_test)

print("随机森林：")
print(precision2)
print(recall2)
print(f12)

# 绘制误差图像

plt.figure(1)
plt.clf()
ax=plt.axes(aspect='equal')
plt.scatter(y_test, y_pred2)
plt.xlabel('True Values')
plt.ylabel('Predictions')
Lims=[0,1]
plt.xlim(Lims)
plt.ylim(Lims)
plt.plot(Lims,Lims)
plt.grid(False)

error = y_pred2 - y_test  
plt.figure(2)
plt.clf()
plt.hist(error,bins=30)
plt.xlabel('Prediction Error')
plt.ylabel('Count')
plt.grid(False)

import os
 
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
tree_graph_dot_path='D:/tree.dot'

from sklearn import tree
from IPython.display import Image

Estimators = forest.estimators_
for index, model in enumerate(Estimators):
    filename = 'RFC_min_sample500' + str(index) + '.pdf'
    dot_data = tree.export_graphviz(model , out_file=None, feature_names=names,
                         class_names='label',
                         filled=True, rounded=True,
                         special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data)
    
#     使用ipython的终端jupyter notebook显示
    Image(graph.create_png())
    graph.write_pdf(filename)
    break

In [38]:
names = ['brand', 'color','cityId','carCode','country','maketype', 'gearbox','oiltype','feature1','feature2',
        'feature3',  'feature5',  'feature6',  'feature8',  'feature9', 'feature10', 'feature11', 'feature14',
        'pushPrice','serial','model','mileage','transferCount','seatings',
               'displacement','newprice','price','feature12-length','feature12-width',
              'feature12-height','feature12-volume','tradeTime-registerDate',
               'tradeTime-licenseDate','tradeTime-modelyear','tradeTime-feature13',
              'pull_pushTime','update_num','update_T_el','delta_price']

# Calculate the importance of variables

random_forest_importance=list(forest.feature_importances_)
random_forest_feature_importance=[(feature,round(importance,8)) 
                                  for feature, importance in zip(names,random_forest_importance)]
random_forest_feature_importance=sorted(random_forest_feature_importance,key=lambda x:x[1],reverse=True)
plt.figure(3)
plt.clf()
importance_plot_x_values=list(range(len(random_forest_importance)))
plt.bar(importance_plot_x_values,random_forest_importance,orientation='vertical')
plt.xticks(importance_plot_x_values,names,rotation='vertical')
plt.xlabel('Variable')
plt.ylabel('Importance')
plt.title('Variable Importances')